In [1]:
import time
import pandas as pd
from wikiapi import WikiApi
wiki = WikiApi()
import re
from gensim.models import Word2Vec
#import word2vec
from scipy.spatial.distance import cosine
import numpy as np
import heapq
import string


In [164]:
cosine(model['chair'], model['sofa']) < cosine(model['chair'], model['char'])

True

In [85]:
def remove_punctuation(s):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in s if ch not in exclude)
    return s
#TODO: fix this or dont use
def get_longword(s, n):
    s = re.split(' ', s)
    return heapq.nlargest(n, s, key=len)

#TODO: make this work with one word/period
def get_avg_vec(words):
	try:
		words = re.split(' ', words)
	except:
		pass
	nwords = 0
	featureVec = np.zeros((num_features,))#,dtype="float32")
	for word in words:
		if word in index2word_set:
			nwords = nwords + 1.
			featureVec = np.add(featureVec,model[word])
	if nwords > 0:
		featureVec = np.divide(featureVec,nwords)
	return featureVec

def get_tvec(row):
    if row['correctAnswer'] == 'A':
        a = row['aavec']
        return a
    elif row['correctAnswer'] == 'B':
        a = row['bavec']
        return a
    elif row['correctAnswer'] == 'C':
        a = row['cavec']
        return a
    elif row['correctAnswer'] == 'D':
        a = row['davec']
        return a
    
def run_through_theta(v):
    return v.dot(theta.as_matrix())

def get_winner_from_avecs(row):
	try:	
		dists = []
		for col in row['answerA':'answerD']:
		#TODO: if 'not' in words: 1-avec
			avec = get_avg_vec(col)
			#if 'not' in re.split(' ',col):
			#	avec = 1-avec
			#	print('NOT')
			#why nan errors?
			if sum(np.isfinite(avec)) < 300:
				print('avec')
				print(avec)
			if  sum(np.isfinite(row['pvec'])) < 300:
				print('pvec')
				print(row['pvec'])
			dist = cosine(avec, row['pvec'])
			dists.append(dist)
		m = np.nanmin(dists)
		best = [i for i, j in enumerate(dists) if j == m]
		if best == [0]:
			return 'A', m, dists
		if best == [1]:

			return 'B' ,m, dists
		if best == [2]:
			return 'C' , m, dists
		if best == [3]:
			return 'D', m, dists
		else:
			print('returning C becuase no best cosine')
			print(len(best))
			print(dists)
			print(m)
			return 'C', m, dists
	except:
		print('some error, returning C')
		return 'C'

In [161]:
data.head()

,id,question,correctAnswer,answerA,answerB,answerC,answerD,keyword,qvec,aavec,...,davec,pvec,closest_avec,avec_dist,dists,dists_avg,akeyword,bkeyword,ckeyword,dkeyword
0,100001,When athletes begin to exercise their heart ra...,C,at the tissue level,at the organ level,at the system level,at the cellular level,"[organization, respiration, coordinate, functi...","[-0.0110091024078, -0.0100927315652, 0.0118642...","[-0.0672742798924, 0.0383596493048, -0.0032832...",...,"[-0.0559718739241, -0.0239021295565, -0.004953...","[-0.0110091024078, -0.0100927315652, 0.0118642...",B,0.673600,"[0.679835183902, 0.673599518912, 0.67691867520...",0.680747,"[tissue, level]","[organ, level]","[system, level]","[cellular, level]"
1,100002,Which example describes a learned behavior in ...,C,smelling the air for odors,barking when disturbed,sitting on command,digging in soil,"[describes, behavior, example, learned, Which]","[0.0217765471898, 0.0255097200454, 0.008364850...","[-0.025296641048, 0.0597655605525, 0.025275064...",...,"[-0.0250138856936, 0.120847150683, 0.053680358...","[0.0217765471898, 0.0255097200454, 0.008364850...",B,0.706429,"[0.72445609405, 0.706428517086, 0.813424001565...",0.764160,"[smelling, odors]","[disturbed, barking]","[sitting, command]","[digging, soil]"
2,100003,When two nuclei are combined into one nucleus ...,D,conversion,reaction,fission,fusion,"[combined, nucleus, release, process, nuclei]","[0.0108336833306, 0.0333348739892, 0.023148844...","[0.0507117174566, 0.0424134396017, -0.03995468...",...,"[-0.0258576832712, 0.00056411215337, 0.1105863...","[0.0108336833306, 0.0333348739892, 0.023148844...",C,0.642293,"[0.751297557367, 0.776472954065, 0.64229285003...",0.707571,[conversion],[reaction],[fission],[fusion]
3,100004,Which is a distinction between an epidemic and...,B,the symptoms of the disease,the geographical area affected,the species of organisms infected,the season in which the disease spreads,"[distinction, epidemic, pandemic, between, Which]","[0.0247666956857, -0.0448009822518, 0.03648750...","[-0.0109115671366, 0.0603187698871, -0.0345962...",...,"[0.0130607932806, 0.0607080888003, -0.03335775...","[0.0247666956857, -0.0448009822518, 0.03648750...",D,0.538173,"[0.570070366424, 0.670986934765, 0.58689460915...",0.591531,"[symptoms, disease]","[geographical, affected]","[organisms, infected]","[disease, spreads]"
4,100005,In which way is the orbit of a comet different...,B,The orbit of Earth is less circular than the o...,The orbit of a comet is more elliptical than t...,The orbital period of Earth is much longer tha...,The orbital period of a comet is more predicta...,"[different, which, orbit, comet, orbit]","[-0.0375164592639, 0.0500212982297, -0.0044289...","[-0.0424050781876, 0.0548852635548, -0.0066005...",...,"[-0.0189889464527, -0.024584245868, -0.0548267...","[-0.0375164592639, 0.0500212982297, -0.0044289...",B,0.209713,"[0.210562819276, 0.209712559749, 0.34429267593...",0.273452,"[circular, orbit]","[elliptical, orbit]","[orbital, orbital]","[predictable, orbital]"


In [3]:
start = time.time()
model = Word2Vec.load_word2vec_format('/Users/liamconnell/Downloads/GoogleNews-vectors-negative300.bin', binary = True)
lap1 = time.time()
print('data gathered: %s' % (lap1 - start))

data gathered: 132.5608139038086


In [4]:
index2word_set = set(model.index2word)
num_features = 300

In [20]:
data = pd.read_csv('../input/training_set.tsv', '\t')
#data  = pd.read_csv('../input/validation_set.tsv', '\t')
#theta = pd.read_csv('../input/theta_100-a_e-10.csv')

In [21]:
data.question = data.question.apply(remove_punctuation)
data.answerA = data.answerA.apply(remove_punctuation)
data.answerB = data.answerB.apply(remove_punctuation)
data.answerC = data.answerC.apply(remove_punctuation)
data.answerD = data.answerD.apply(remove_punctuation)

In [24]:
data_full = data.copy()
data = data_full[:2000]

In [124]:
#theta = pd.read_csv('../input/qkeywd_theta_100-a_e-10.csv')

In [27]:
theta = pd.DataFrame(np.identity(300))

In [147]:
data['keyword'] =data['question'].apply(get_longword, args = (5,))

#I get to choose this
data['qvec'] = data.keyword.apply(get_avg_vec) 


/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [146]:
#data['qvec'] = data.question.apply(get_avg_vec) 
data['akeyword'] =data['answerA'].apply(get_longword, args = (2,))
data['bkeyword'] =data['answerB'].apply(get_longword, args = (2,))
data['ckeyword'] =data['answerC'].apply(get_longword, args = (2,))
data['dkeyword'] =data['answerD'].apply(get_longword, args = (2,))

data['aavec'] = data['akeyword'].apply(get_avg_vec)
data['bavec'] = data['bkeyword'].apply(get_avg_vec)
data['cavec'] = data['ckeyword'].apply(get_avg_vec)
data['davec'] = data['dkeyword'].apply(get_avg_vec)

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [26]:
data['aavec'] = data['answerA'].apply(get_avg_vec)
data['bavec'] = data['answerB'].apply(get_avg_vec)
data['cavec'] = data['answerC'].apply(get_avg_vec)
data['davec'] = data['answerD'].apply(get_avg_vec)

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [148]:
data['pvec'] = data.qvec.apply(run_through_theta)

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [149]:
data['closest_avec'] = data.apply(get_winner_from_avecs, axis=1)


returning C becuase no best cosine
4
[0.61409244070064983, 0.61409244070064983, 0.61409244070064983, 0.61409244070064983]
0.614092440701
returning C becuase no best cosine
2
[0.83706870344266837, 0.88607881670466182, 0.83706870344266837, 0.88607881670466182]
0.837068703443
returning C becuase no best cosine
4
[0.75547675131548597, 0.75547675131548597, 0.75547675131548597, 0.75547675131548597]
0.755476751315
returning C becuase no best cosine
2
[0.4062484245604322, 0.42175795990451315, 0.42175795990451315, 0.4062484245604322]
0.40624842456
returning C becuase no best cosine
2
[0.52525205085788063, 0.52525205085788063, 0.58484763437827947, 0.53998686886397407]
0.525252050858
returning C becuase no best cosine
2
[0.5113047264668642, 0.53417197715656362, 0.29634133758760273, 0.29634133758760273]
0.296341337588
returning C becuase no best cosine
2
[0.55914201813483377, 0.48895236352320237, 0.55914201813483377, 0.48895236352320237]
0.488952363523
returning C becuase no best cosine
4
[0.56826

/Users/liamconnell/anaconda/lib/python3.4/site-packages/numpy/lib/nanfunctions.py:227: RuntimeWarning: All-NaN axis encountered
  warnings.warn("All-NaN axis encountered", RuntimeWarning)
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':



0
[nan, nan, nan, nan]
nan


In [150]:
data['avec_dist'] = [x[1] for x in data['closest_avec']]
data['dists'] = [x[2] for x in data['closest_avec']]
data['closest_avec'] = [x[0] for x in data['closest_avec']]

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [151]:
data['dists_avg'] = [np.mean(x) for x in data.dists]

/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [152]:
results_correct = data[data['correctAnswer'] == data['closest_avec']]
results_incorrect = data[data['correctAnswer'] != data['closest_avec']]

In [153]:
print(len(results_correct))
print(len(results_incorrect))
len(results_correct)/len(data)

607
1393


0.3035

In [137]:
def numwords(s):
    s = re.split(' ', s)
    return len(s)

def specs(results):
    nw_q = np.mean(results.question.apply(numwords))
    print('length of question: %s' %nw_q)
    nw_a = (np.mean(results.answerA.apply(numwords))+np.mean(results.answerB.apply(numwords)) +
            np.mean(results.answerC.apply(numwords))+np.mean(results.answerD.apply(numwords)))/4
    print('length of answers: %s' %nw_a)
    oneword = len(results[results['answerA'].apply(numwords) ==1])/len(results)
    print('percent of one word answers: %s' 
          %oneword)
    vec_dist = np.mean(results.avec_dist)
    print('average distance: %s' % vec_dist)
    dif1 = np.mean(results['dists_avg'] - results.avec_dist)
    print('choice dif minus avg dif: %s'% dif1)
    std = np.nanmean([np.std(x) for x in results.dists])
    print(std)
    print('\t')

In [145]:
specs(results_correct)
specs(results_incorrect)

length of question: 21.70719178082192
length of answers: 4.808219178082192
percent of one word answers: 0.24315068493150685
average distance: 0.5786094637234608
choice dif minus avg dif: 0.07027791672226034
0.0540180744819
	
length of question: 22.189265536723163
length of answers: 5.320444915254237
percent of one word answers: 0.21045197740112995
average distance: 0.5689335422495899
choice dif minus avg dif: 0.06672336460665429
0.0521196408633
	


In [149]:
#sample = pd.read_csv('../input/sample_submission.csv')
#sample['correctAnswer'] = data['closest_avec']
#sample.to_csv('../output/qkeywd-5_theta-id.csv', index=False)